### Import MySQL Libraries

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
import datetime as dt

In [2]:
%store -r HOST
%store -r USER
%store -r PASS

In [3]:
host=HOST
user = USER
password = PASS

### Connecting to MySQL Server

In [13]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
        host=host_name,
        user=user_name,
        passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

### Creating a New Databse

In [14]:
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")

### Connecting to the Database

In [15]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
        host=host_name,
        user=user_name,
        passwd=user_password,
        database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

### Creating a Query Execution Function

In [16]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

### Reading Data

In [17]:
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

### Creating Records from Lists

In [18]:
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [19]:

connection = create_server_connection(host, user, password)
create_database(connection, "CREATE DATABASE IF NOT EXISTS music;")

connection.close()

MySQL Database connection successful
Database created successfully


### Creating the Channel Table

In [20]:
connection = create_db_connection(host,user,password,'music')
create_music_table = """ CREATE TABLE channels (
                            channel_id varchar(255) PRIMARY KEY,
                            channel_name varchar(255),
                            viewers bigint,
                            subscribers bigint,
                            published date); """

MySQL Database connection successful


In [21]:
execute_query(connection,create_music_table)

Error: '1050 (42S01): Table 'channels' already exists'


### Creating the Video Table

In [22]:
create_vid_table = """ CREATE TABLE videos (
                            video_id varchar(255) PRIMARY KEY,
                            artist varchar(255),
                            title varchar(255),
                            likes bigint,
                            dislikes bigint,
                            views bigint,
                            comments bigint,
                            channel_id varchar(255),
                            FOREIGN KEY (channel_id) REFERENCES channels(channel_id)); """

In [23]:
execute_query(connection,create_vid_table)

Error: '1050 (42S01): Table 'videos' already exists'


In [24]:
channels = pd.read_csv("channels.csv",index_col=0)

In [25]:
channels.head()

,channelId,channel_name,viewers,subscribers,published,genre
0,UCTbOjJv4X1fk-LdfHnWAK1Q,Daniel Powter,243956424,32700,2006-04-29T04:06:19Z,pop
1,UCkdc7gHpavxpgGalxKbgSHA,Sean Paul,2157186944,3810000,2009-04-23T20:28:05Z,pop
2,UC5Y-gAeGpzgzu0ObzKnhPFA,Nelly Furtado,1849529691,1800000,2006-11-04T00:41:49Z,latin
3,UCUg8KhMXhFxJm-YQzOQ7nPw,James Blunt,1619839415,2700000,2006-08-30T13:08:54Z,pop
4,UCYLNGLIzMhRTi6ZOLjAPSmw,Shakira,21773623218,34400000,2005-10-16T09:08:20Z,pop


### Convert Published Date Column to a Date Type

In [26]:
channels['published']=pd.to_datetime(channels['published'])

In [27]:
channels['published']=pd.to_datetime(channels['published'],format="%Y-%m-%d")

In [28]:
channels['published']=channels['published'].dt.date

In [29]:
channels.head()

,channelId,channel_name,viewers,subscribers,published,genre
0,UCTbOjJv4X1fk-LdfHnWAK1Q,Daniel Powter,243956424,32700,2006-04-29,pop
1,UCkdc7gHpavxpgGalxKbgSHA,Sean Paul,2157186944,3810000,2009-04-23,pop
2,UC5Y-gAeGpzgzu0ObzKnhPFA,Nelly Furtado,1849529691,1800000,2006-11-04,latin
3,UCUg8KhMXhFxJm-YQzOQ7nPw,James Blunt,1619839415,2700000,2006-08-30,pop
4,UCYLNGLIzMhRTi6ZOLjAPSmw,Shakira,21773623218,34400000,2005-10-16,pop


In [30]:
genres=pd.read_csv("genres.csv",index_col=0)

In [31]:
genres.head()

,artist,title,year,lyrics,words,genre
0,Daniel Powter,Bad Day,2006,where is the moment we needed the most?you kic...,"['moment', 'needed', 'mostyou', 'kick', 'leaf'...",pop
1,Sean Paul,Temperature,2006,"the gyal dem schillaci, sean a paulso me give ...","['gyal', 'dem', 'schillaci', 'sean', 'paulso',...",pop
2,Nelly Furtado,Promiscuous,2006,nelly furtado &timbalandam i throwin' you off?...,"['nelly', 'furtado', 'timbalandam', 'throwin',...",latin
3,James Blunt,Youre Beautiful,2006,"my life is brilliantmy life is brilliant, my l...","['life', 'brilliantmy', 'brilliant', 'love', '...",pop
4,Shakira,Hips Dont Lie,2006,wyclef jeanladies up in here tonightno fightin...,"['wyclef', 'jeanladies', 'tonightno', 'fightin...",pop


In [144]:
video = pd.read_csv("video.csv",index_col=0)

In [145]:
video

,video_id,artist,title,likes,dislikes,views,comments,published
0,gH476CxJxfg,Daniel Powter,Daniel Powter - Bad Day (Official Music Video)...,1534284,23998,198275400,51550,2009-10-27T02:01:38Z
1,BbtcTb_WMsg,Daniel Powter,Daniel Powter - Love You Lately (Official Musi...,20718,397,5475397,687,2009-10-27T02:00:07Z
2,vEY_mg2y-rg,Daniel Powter,Daniel Powter - Free Loop (Official Music Video),89155,1362,12486084,3101,2009-10-27T01:47:35Z
3,oDAqDLmixrw,Daniel Powter,Save Your Life - Ayaka Hirahara & Daniel Powter,5164,50,626438,256,2021-02-02T15:00:13Z
4,5E4j_n17cTI,Daniel Powter,Daniel Powter - Next Plane Home (Video),13046,320,3640584,520,2009-10-29T21:24:41Z
...,...,...,...,...,...,...,...,...
8939,m8rM7Tox1HE,Lewis Capaldi,Lewis Capaldi - Grace,273709,5655,39374455,6880,2018-09-21T10:00:07Z
8940,sB8n58FHBxw,Lewis Capaldi,Lewis Capaldi - Bruises (Official Live Video),122915,2097,12831682,1930,2017-04-12T06:00:03Z
8941,PUZqMw4rkVs,Lewis Capaldi,Lewis Capaldi - Lost On You (Live),268949,3235,20180526,3489,2017-07-28T11:00:02Z
8942,oyBUgOFlBU4,Lewis Capaldi,Lewis Capaldi - Fade (Live),24233,243,1839017,674,2017-11-17T00:00:02Z


In [146]:
video.head()

,video_id,artist,title,likes,dislikes,views,comments,published
0,gH476CxJxfg,Daniel Powter,Daniel Powter - Bad Day (Official Music Video)...,1534284,23998,198275400,51550,2009-10-27T02:01:38Z
1,BbtcTb_WMsg,Daniel Powter,Daniel Powter - Love You Lately (Official Musi...,20718,397,5475397,687,2009-10-27T02:00:07Z
2,vEY_mg2y-rg,Daniel Powter,Daniel Powter - Free Loop (Official Music Video),89155,1362,12486084,3101,2009-10-27T01:47:35Z
3,oDAqDLmixrw,Daniel Powter,Save Your Life - Ayaka Hirahara & Daniel Powter,5164,50,626438,256,2021-02-02T15:00:13Z
4,5E4j_n17cTI,Daniel Powter,Daniel Powter - Next Plane Home (Video),13046,320,3640584,520,2009-10-29T21:24:41Z


In [147]:
video['channel_id']=None
for i in range(len(channels)):
    for j in range(len(video)):
        if channels['channel_name'][i] == video['artist'][j]:
            video['channel_id'][j] = channels['channelId'][i]

<ipython-input-147-f72d5d254180>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  video['channel_id'][j] = channels['channelId'][i]


In [148]:
video.head(50)

,video_id,artist,title,likes,dislikes,views,comments,published,channel_id
0,gH476CxJxfg,Daniel Powter,Daniel Powter - Bad Day (Official Music Video)...,1534284,23998,198275400,51550,2009-10-27T02:01:38Z,UCTbOjJv4X1fk-LdfHnWAK1Q
1,BbtcTb_WMsg,Daniel Powter,Daniel Powter - Love You Lately (Official Musi...,20718,397,5475397,687,2009-10-27T02:00:07Z,UCTbOjJv4X1fk-LdfHnWAK1Q
2,vEY_mg2y-rg,Daniel Powter,Daniel Powter - Free Loop (Official Music Video),89155,1362,12486084,3101,2009-10-27T01:47:35Z,UCTbOjJv4X1fk-LdfHnWAK1Q
3,oDAqDLmixrw,Daniel Powter,Save Your Life - Ayaka Hirahara & Daniel Powter,5164,50,626438,256,2021-02-02T15:00:13Z,UCTbOjJv4X1fk-LdfHnWAK1Q
4,5E4j_n17cTI,Daniel Powter,Daniel Powter - Next Plane Home (Video),13046,320,3640584,520,2009-10-29T21:24:41Z,UCTbOjJv4X1fk-LdfHnWAK1Q
5,JLkuTQMaHYg,Daniel Powter,Daniel Powter - Lie To Me (Video),2723,123,1402462,148,2009-10-27T01:56:02Z,UCTbOjJv4X1fk-LdfHnWAK1Q
6,mhnn9TZTE3w,Daniel Powter,Daniel Powter - Jimmy Gets High (Video),5639,161,1670905,340,2009-10-27T01:54:35Z,UCTbOjJv4X1fk-LdfHnWAK1Q
7,J4dagWIRd48,Daniel Powter,Daniel Powter - Cupid,20598,428,4010567,886,2012-05-29T07:00:10Z,UCTbOjJv4X1fk-LdfHnWAK1Q
8,1CaMX1U9Qvo,Daniel Powter,Daniel Powter - Crazy All My Life (Official M...,5337,164,663353,150,2013-03-27T18:48:40Z,UCTbOjJv4X1fk-LdfHnWAK1Q
9,E2hpitcq1G8,Daniel Powter,T.U.B.S (陈情少年) 'The Beauty of Following Our He...,19094,59,456753,921,2021-01-07T05:00:07Z,UCTbOjJv4X1fk-LdfHnWAK1Q


In [32]:
channel_ids = []
channel_name = []
channel_views = []
channel_subscribers = []
channel_published = []
for i in range(len(channels)):
    channel_ids.append(channels['channelId'][i])
    channel_name.append(channels['channel_name'][i])
    channel_views.append(channels['viewers'][i])
    channel_subscribers.append(channels['subscribers'][i])
    channel_published.append(channels['published'][i])

### Inserting Rows into the Channel Table

In [33]:
sql= '''INSERT INTO channels(channel_id,channel_name,viewers,subscribers,published) VALUES (%s,%s,%s,%s,%s)'''

records = []

for i in range(len(channels)):
    records.append([channel_ids[i],channel_name[i],np.int64(channel_views[i]).item(),np.int64(channel_subscribers[i]).item(),channel_published[i]])

#Pandas stores the integer as numpy.int64 or numpy.int32 which MySQL does not support
#So the values must be converted to base int using the np.int64().item()
conenction = create_db_connection(host,user,password,'music')
execute_list_query(connection,sql,records)
connection.close()
    
    


MySQL Database connection successful
Error: '1062 (23000): Duplicate entry 'UCTbOjJv4X1fk-LdfHnWAK1Q' for key 'channels.PRIMARY''


In [150]:
video.drop(video.loc[video['video_id']=='#NAME?'].index,inplace=True)

In [151]:
video.drop_duplicates(subset=['video_id'],inplace=True)

In [152]:
video.reset_index(inplace=True)

In [153]:
video_ids = []
video_artist = []
video_title = []
video_likes = []
video_dislikes = []
video_views = []
video_comments = []
video_channel = []
for i in range(len(video)):
    video_ids.append(video['video_id'][i])
    video_artist.append(video['artist'][i])
    video_title.append(video['title'][i])
    video_likes.append(video['likes'][i])
    video_dislikes.append(video['dislikes'][i])
    video_views.append(video['views'][i])
    video_comments.append(video['comments'][i])
    video_channel.append(video['channel_id'][i])

In [154]:
len(video_ids)

8805

### Inserting Rows Into the Video Table

In [155]:
sql= '''INSERT INTO videos(video_id,artist,title,likes,dislikes,views,comments,channel_id) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'''

records = []

for i in range(len(video)):
    records.append([video_ids[i],video_artist[i],video_title[i],np.int64(video_likes[i]).item(),np.int64(video_dislikes[i]).item(),np.int64(video_views[i]).item(),np.int64(video_comments[i]).item(),video_channel[i]])

    
conenction = create_db_connection(host,user,password,'music')
execute_list_query(connection,sql,records)
connection.close()

MySQL Database connection successful
Query successful
